In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import gurobipy as gp
from gurobipy import GRB
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from scipy.interpolate import griddata

In [5]:
# Arizona shapefile

us_states = gpd.read_file('cb_2018_us_state_500k/cb_2018_us_state_500k.shp')
arizona = us_states[us_states['NAME'] == 'Arizona']
arizona.to_crs("EPSG:5070", inplace=True) # we use EPSG 5070 to get more accurate area measurements. https://epsg.io/5070-1252

# water basins shapefile

huc_subbasins = gpd.read_file('HUC8_CONUS/HUC8_US.shp') # read in subbasins
huc_subbasins['has_arizona'] = huc_subbasins['STATES'].map(lambda x: 'AZ' in x)
huc_subbasins_az = huc_subbasins[huc_subbasins['has_arizona']]
huc_subbasins_az.to_crs("EPSG:5070", inplace=True)

huc_subbasins_az['HUC6'] = huc_subbasins_az['HUC8'].map(lambda x: x[:6])

# also, in case we want centroid:
# huc_subbasins_az['centroid'] = huc_subbasins_az.to_crs("+proj=cea").centroid.to_crs(huc_subbasins_az.crs)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarni

In [6]:
# get lists of HUC8 subbasins that group into HUC6 basins
huc6_huc8_dict = dict()

for huc6 in huc_subbasins_az['HUC6'].unique():
    huc_filter = huc_subbasins_az[huc_subbasins_az['HUC6'] == huc6]
    huc6_huc8_dict[huc6] = huc_filter['HUC8'].to_list()

# data values for HUC8 regions generated from the paper by Abu Bakar Siddik et al 2021: https://iopscience.iop.org/article/10.1088/1748-9326/abfba1?_sp=b48260d8-0a7b-4784-9d4b-0e1ac60ee727
water_carbon = pd.read_excel("14504913/SI_XLS/Results.xlsx", sheet_name="Table 3", skiprows=1)

water_carbon['HUC8_str'] = water_carbon['HUC8'].map(lambda x: ''.join(['0']*(8-len(str(x)))) + str(x))

# filter to only HUC rows
water_carbon['not_nan'] = ~water_carbon['WSF_1MW_DC'].isna()
water_carbon = water_carbon[water_carbon['not_nan']]

# now merge with the arizona HUC data
huc_subbasins_az_join = huc_subbasins_az.merge(water_carbon, how="inner", left_on="HUC8", right_on="HUC8_str")
huc_subbasins_az_join = gpd.clip(huc_subbasins_az_join, arizona)

In [7]:
# take a look at water stress factor, carbon factor data
wsf_dict = {k: v for k, v in zip(huc_subbasins_az_join['HUC8_y'], huc_subbasins_az_join['WSF_1MW_DC'])}
cf_dict = {k: v for k, v in zip(huc_subbasins_az_join['HUC8_y'], huc_subbasins_az_join['CF_1MW_DC'])}

s_max = huc_subbasins_az_join['WSF_1MW_DC'].max()
e_max = huc_subbasins_az_join['CF_1MW_DC'].max()

In [ ]:
# estimations of existing data centers: https://www.datacentermap.com/
# power of existing data centers? Unclear. https://baxtel.com/data-center/phoenix
# well, in total there is around 602.8 MW in the Phoenix area. https://azbigmedia.com/real-estate/phoenix-ranks-4th-among-north-american-data-center-markets/
# and, in the Phoenix area there are 124 data centers. https://www.datacentermap.com/usa/arizona/
# so, let's say on average, each existing data center takes on 5 MW of load.

# existing inventory HUC8
# 15050301: 6
# 15050302: 2
# 15050303: 4 (from Eloy) 
# 15050100: 17 (from Mesa airport AZA) 13 (from Chandler airport CHD) 4 (from Gilbert)
# 15060106: 2 (from Mesa) 25 (from Phoenix airport and Scottsdale) 2 (from upper Scottsdale)
# 15070102: 6 (from Deer Valley) 40 (from Litchfield park airport)
# 15070103: 1 (from Buckeye/Arlington)


# totals (done by hand, because data lies behind paywalls):
# 15050301: 6
# 15050302: 2
# 15050303: 4
# 15050100: 34
# 15060106: 29
# 15070102: 46
# 15070103: 1

In [8]:
# electricity prices
# https://www.energysage.com/local-data/electricity-cost/az/
# cochise county: 16 cents/kwh
# coconino county: 15 cents/kwh
# gila county: 20 cents/kwh
# la paz county: 15 cents/kwh
# maricopa county: 15 cents/kwh
# mohave county: 18 cents/kwh
# navajo county: 15 cents/kwh
# pima county: 18 cents/kwh
# pinal county: 16 cents/kwh
# yavapai county: 15 cents/kwh
# yuma county: 15 cents/kwh

us_counties = gpd.read_file('cb_2018_us_county_500k/cb_2018_us_county_500k.shp') # read in counties
arizona_counties = us_counties[us_counties['STATEFP'] == '04'] # filter to arizona
arizona_counties.to_crs("EPSG:5070", inplace=True)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [9]:
# units: cents/kwh
elec_price_dict = {'Cochise': 16, 'Coconino': 15, 'Gila': 20, 'La Paz': 15, 'Maricopa': 15, 'Mohave': 18, 
                   'Navajo': 15, 'Pima': 18, 'Pinal': 16, 'Yavapai': 15, 'Yuma': 15}

# new units: dollars/mw = cents/kwh * (0.01 dollar/1 cent) * (1000 kwh/1 mw)
elec_price_dict = {k: 10*v for k, v in elec_price_dict.items()}

has_elec_price = set(elec_price_dict.keys())
num_elec_price = len(has_elec_price)

# find arizona counties with electricity price data
arizona_counties['has_elec_price'] = arizona_counties['NAME'].map(lambda x: x in has_elec_price)
arizona_counties_price = arizona_counties[arizona_counties['has_elec_price']]

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [ ]:
# assign electricity price in HUC8 regions to closest county with price
# consider: if desired, we can add some random noise

# build up the elec_price ($/MW) column
huc8_codes = set(huc_subbasins_az_join['HUC8_y'])

for code in huc8_codes:
    huc8_row = huc_subbasins_az_join[huc_subbasins_az_join['HUC8_y'] == code]
    county_intersect = arizona_counties_price['geometry'].intersection(huc8_row['geometry'].iloc[0])
    
    area_sum = np.sum(county_intersect.area)

    if area_sum > 0: # take a weighted sum based on area of intersection with counties
        weighted_price = 0
        for idx, val in county_intersect.area.items(): 
            weighted_price += elec_price_dict[arizona_counties_price.loc[idx, 'NAME']]*val
        weighted_price /= area_sum
    else: # just find the closest one
        county_dist = arizona_counties_price['geometry'].distance(huc8_row['geometry'].iloc[0])
        county_dist.sort_values(ascending=True, inplace=True)
        weighted_price = elec_price_dict[arizona_counties_price.loc[county_dist.index[0], 'NAME']]

In [ ]:
# major cities for baseline data centers
# basically, atm all of the data centers are in these three, so im approximating by just using their central lat/lon
# count is how the 129 are split among the locations
CITIES = {
    "Phoenix": {"lat": 33.4484, "lng": -112.0740, "count": 121},
    "Tucson": {"lat": 32.2226, "lng": -110.9747, "count": 7},
    "Nogales": {"lat": 31.3405, "lng": -110.9420, "count": 1}
}

# create the baseline data centers locations
def create_baseline_centers():
    centers = []
    center_id = 0

    for city, info in CITIES.items():
        # decided to add a slight random variation to prevent all centers being at exactly the same point
        for i in range(info["count"]):
            #within ~5km of the city center
            lat_variation = (np.random.random() - 0.5) * 0.05
            lng_variation = (np.random.random() - 0.5) * 0.05

            centers.append({
                "id": center_id,
                "city": city,
                "lat": info["lat"] + lat_variation,
                "lng": info["lng"] + lng_variation
            })
            center_id += 1

    return centers

# here, i made a grid of potential locations across the approximate Arizona
def create_location_grid(resolution=0.2):
    locations = []
    loc_id = 0
    for lat in np.arange(AZ_MIN_LAT, AZ_MAX_LAT + resolution, resolution):
        for lng in np.arange(AZ_MIN_LNG, AZ_MAX_LNG + resolution, lng_spacing): # should this be resolution rather than lng_spacing? - RICHARD
            locations.append({
                "id": loc_id,
                "lat": lat,
                "lng": lng
            })
            loc_id += 1

    return locations

# generate water stress data based on the Aqueduct data approximately
# will have to request the actual shapefiles for later on
# so this is just simplified, with higher values = more stress (0-5 scale)
def generate_water_stress(locations):

    # northern Arizona: lower stress (more precipitation)
    # central & Phoenix metro: high stress
    # southern Arizona: very high stress
    # western Arizona along colorado river: medium stress

    for loc in locations:
        lat, lng = loc["lat"], loc["lng"]
        base_stress = 5.0 - (lat - AZ_MIN_LAT) / (AZ_MAX_LAT - AZ_MIN_LAT) * 2.5

        # phoenix metro area: very high stress
        if (33.0 <= lat <= 34.0) and (-112.5 <= lng <= -111.5):
            stress = 4.5 + np.random.random() * 0.5  # 4.5-5.0

        # tucson area: very high stress
        elif (31.9 <= lat <= 32.5) and (-111.2 <= lng <= -110.7):
            stress = 4.2 + np.random.random() * 0.8  # 4.2-5.0

        # colorado river region: medium stress due to water access
        elif lng < -113.5:
            stress = 2.5 + np.random.random() * 1.5  # 2.5-4.0

        # northern mountains (e.g., flagstaff): lower stress
        elif (34.5 <= lat <= 36.0) and (-112.5 <= lng <= -111.0):
            stress = 1.8 + np.random.random() * 1.2  # 1.8-3.0

        #  northeastern AZ (like the navajo area): medium-high stress
        elif lat > 35.0 and lng > -110.5:
            stress = 3.2 + np.random.random() * 1.0  # 3.2-4.2

        # if not near, we can just use base stress with some noise
        else:
            stress = base_stress + (np.random.random() - 0.5) * 1.0

        stress = max(0, min(5, stress))
        loc["water_stress"] = stress

    return locations

#electricity price data by county

# out of curiosity, where are the electricity prices sourced from? In case we want to reproduce
# these electricity price calculations for other states, and I can help - RICHARD
def generate_electricity_prices(locations):

    # note: the average commercial electricity price in Arizona is ~$0.0811/kWh (8.11 cents)

    for loc in locations:
        lat, lng = loc["lat"], loc["lng"]
        base_price = 0.0811

        # phoenix metro area (APS & SRP territory)
        if (33.0 <= lat <= 34.0) and (-112.5 <= lng <= -111.5):
            if lng < -112.0:  # SRP territory
                price = base_price * (0.92 + np.random.random() * 0.08)  # 8% lower on average
            else:  # APS territory
                price = base_price * (0.98 + np.random.random() * 0.07)  # 2% lower on average

        # tucson area (tucson electric power)
        elif (31.9 <= lat <= 32.5) and (-111.2 <= lng <= -110.7):
            price = base_price * (1.03 + np.random.random() * 0.07)  # 3-10% higher

        # rural areas (higher distribution costs)
        elif lat > 35.0 or lat < 31.7 or lng < -113.0:
            price = base_price * (1.10 + np.random.random() * 0.15)  # 10-25% higher

        # otherwise, i use base price with some noise
        else:
            price = base_price * (0.95 + np.random.random() * 0.15)  # ±10% variation

        loc["electricity_price"] = price

    return locations

# carbon emission rate data in tons Co2/'Mwh
def generate_carbon_emissions(locations):

    # avg emissions for arizona's grid: ~0.4 tons CO2/MWh (400 kg/MWh)
    # from what i looked up, Palo Verde Nuclear Plant creates the biggest variation
    # there's probably a better way to do this, but for the approximation, I just use the plant to vary this val plus some of the predominant plant type variation otherwise

    for loc in locations:
        lat, lng = loc["lat"], loc["lng"]

        base_emissions = 0.40  # tons CO2 per MWh

        # areas near Palo Verde Nuclear Generating Station (west of phoenix)
        if (33.2 <= lat <= 33.5) and (-113.0 <= lng <= -112.5):
            emissions = base_emissions * (0.65 + np.random.random() * 0.10)  # 25-35% lower

        # phoenix metro area (mixed generation)
        elif (33.0 <= lat <= 34.0) and (-112.5 <= lng <= -111.5):
            emissions = base_emissions * (0.90 + np.random.random() * 0.15)  # 0-25% lower

        # northeastern AZ (coal plants)
        elif lat > 35.0 and lng > -110.5:
            emissions = base_emissions * (1.20 + np.random.random() * 0.15)  # 20-35% higher

        # southern AZ (more solar penetration)
        elif lat < 32.5:
            emissions = base_emissions * (0.85 + np.random.random() * 0.15)  # 0-30% lower

        else:
            emissions = base_emissions * (0.90 + np.random.random() * 0.20)  # ±15% variation

        loc["carbon_emissions"] = emissions

    return locations

# water-use efficiency scores
def generate_water_efficiency(locations):
    # based on climate factors like temperature, humidity, and technology potential

    for loc in locations:
        lat, lng = loc["lat"], loc["lng"]

        # it seems that higher latitudes and elevations typically have better efficiency due to cooler climate, so i just came up with this
        base_efficiency = 3.0 + (lat - AZ_MIN_LAT) / (AZ_MAX_LAT - AZ_MIN_LAT) * 2.0

        # hot desert areas (e.g., phoenix, yuma) will have poor efficiency
        if ((33.0 <= lat <= 34.0) and (-112.5 <= lng <= -111.5)) or (lng < -113.5 and lat < 33.0):
            efficiency = 1.5 + np.random.random() * 1.0  # 1.5-2.5 (poor)

        #\higher elevation areas ( e.g., flagstaff, high country) - good efficiency
        elif ((35.0 <= lat <= 36.0) and (-112.5 <= lng <= -111.0)):
            efficiency = 4.0 + np.random.random() * 1.0  # 4.0-5.0 (excellent)

        # mountain transition zones will have moderate to good efficiency
        elif ((34.0 <= lat <= 35.0) and (-112.5 <= lng <= -111.0)):
            efficiency = 3.0 + np.random.random() * 1.5  # 3.0-4.5 (good)

        else:
            efficiency = base_efficiency + (np.random.random() - 0.5) * 1.0
        efficiency = max(1, min(5, efficiency))
        loc["water_efficiency"] = efficiency

    return locations

# distance function to calculate distances between locations (in km)
# lowkey just took this from my old UROP code, idrk if its correct lol
def calc_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers

    # Convert latitude and longitude from degrees to radians
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)

    # Difference in coordinates
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = np.sin(dlat/2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    distance = R * c

    return distance

# REPLACEMENT - RICHARD, taken from here: https://gis.stackexchange.com/questions/425452/calculate-distance-between-two-lat-lon-alt-points-in-python
# from pyproj import Geod
# g = Geod(ellps="WGS84")
# def calc_distance_alt(lat1, lon1, lat2, lon2):
#   _, _, distance = g.inv(lon1, lat1, lon2, lat2)
#   return distance

# check if a lcation is within arizona's boundaries
# this is simplified, w/ real implementation i think we should use a proper shape file
# no problem! I should be able to do that in a standard manner, with geopandas. Example: https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.contains.html#geopandas.GeoSeries.contains - RICHARD
def is_in_arizona(lat, lng):
  # basically i made it a rectangle
    return (AZ_MIN_LAT <= lat <= AZ_MAX_LAT) and (AZ_MIN_LNG <= lng <= AZ_MAX_LNG)

# optimization function
def optimize_data_center_locations(baseline_centers, potential_locations, min_distance=10):
    print(f"Starting optimization with {len(baseline_centers)} data centers and {len(potential_locations)} potential locations")
    np.random.seed(42)

    locations = generate_water_stress(potential_locations)
    locations = generate_electricity_prices(locations)
    locations = generate_carbon_emissions(locations)
    locations = generate_water_efficiency(locations)

    location_ids = [loc["id"] for loc in locations]
    water_stress = {loc["id"]: loc["water_stress"] for loc in locations}
    electricity_price = {loc["id"]: loc["electricity_price"] for loc in locations}
    carbon_emissions = {loc["id"]: loc["carbon_emissions"] for loc in locations}
    water_efficiency = {loc["id"]: loc["water_efficiency"] for loc in locations}

    # coords for distance calculations
    location_coords = {loc["id"]: (loc["lat"], loc["lng"]) for loc in locations}

    # normalization values for the obj function
    max_water_stress = max(water_stress.values())
    max_electricity_price = max(electricity_price.values())
    max_carbon_emissions = max(carbon_emissions.values())
    min_water_efficiency = min(water_efficiency.values())
    max_water_efficiency = max(water_efficiency.values())

    # objective weights, which can be adjusted based on priorities (kind of chose randomly, so if you want to update at any point, feel free :) )
    # sounds great! - RICHARD
    # my initial reaction is to set water efficiency weight to zero lol.  but i definitely have some biases/leanings, so we can definitely discuss this. - RICHARD
    # another interesting question, kind of taking from my UROP last year, is: at what threshold of objective weights, e.g. on water, do we need before it becomes - RICHARD
    # unreasonable to place data centers in hot, desert/dry environments? - RICHARD
    # I suppose we can try a few different weightings. (water stress, elec, carbon, water efficiency) = (1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0), (0, 0, 0, 1), (0.35, 0.25, 0.25, 0.15)
    weights = {
        "water_stress": 0.35,
        "electricity_price": 0.25,
        "carbon_emissions": 0.25,
        "water_efficiency": 0.15
    }

    model = gp.Model("DataCenterOptimization")

    # decision variables: x[d,l] = 1 if data center d is placed at location l
    x = {}
    for d in range(len(baseline_centers)):
        for l in location_ids:
            x[d, l] = model.addVar(vtype=GRB.BINARY, name=f"x_{d}_{l}")

    # minimize weighted combination of metrics
    obj = gp.LinExpr()

    for d in range(len(baseline_centers)):
        for l in location_ids:
            # note: for water efficiency, higher is better, so we invert it
            # out of curiosity, why not 1 - water_efficiency[l]/max_water_efficiency? But works either way! - RICHARD
            normalized_water_efficiency = 1 - (water_efficiency[l] - min_water_efficiency) / (max_water_efficiency - min_water_efficiency)

            norm_water_stress = water_stress[l] / max_water_stress
            norm_electricity = electricity_price[l] / max_electricity_price
            norm_emissions = carbon_emissions[l] / max_carbon_emissions


            # I believe that relocating a data center costs around $120,000: https://www.google.com/url?q=https://apposite-tech.com/best-practices-data-center-relocation/&sa=D&source=docs&ust=1744170960046669&usg=AOvVaw1nspqunGp5e_Gn6C7xgrJB. - RICHARD
            # Perhaps we could also induce a cost for relocating a data center. So you are more reluctant to actually move a data center unless absolutely necessary. - RICHARD

            weighted_sum = (
                weights["water_stress"] * norm_water_stress +
                weights["electricity_price"] * norm_electricity +
                weights["carbon_emissions"] * norm_emissions +
                weights["water_efficiency"] * normalized_water_efficiency
            )

            obj += x[d, l] * weighted_sum

    model.setObjective(obj, GRB.MINIMIZE)

    # each data center must be placed at exactly one location
    for d in range(len(baseline_centers)):
        model.addConstr(gp.quicksum(x[d, l] for l in location_ids) == 1, f"one_loc_per_dc_{d}")

    #each location can have at most one data center
    for l in location_ids:
        model.addConstr(gp.quicksum(x[d, l] for d in range(len(baseline_centers))) <= 1, f"one_dc_per_loc_{l}")

    # idk if we need this, but its basically setting a minimum distance between data centers since i dont want the model spitting them out on top of eachy other
    # Ooo great point. I feel like this gets at our conversation about why you would ever consider clustering data centers. - RICHARD
    # However, I wonder if the water scarcity aspect will naturally cause the model to not cram all the data centers together, if they
    # start to pile on too much burden on the local water system. - RICHARD
    # Anyhow, I believe the fact that you set the locations in a grid pattern at least gives some minimum spacing. We can keep talking
    # about this more if needed! - RICHARD
    if min_distance > 0:
        location_distances = {}
        for l1 in location_ids:
            for l2 in location_ids:
                if l1 < l2:
                    lat1, lng1 = location_coords[l1]
                    lat2, lng2 = location_coords[l2]
                    dist = calc_distance(lat1, lng1, lat2, lng2)
                    location_distances[(l1, l2)] = dist
                    location_distances[(l2, l1)] = dist

        # so basically, for locations that are too close in lat/lon, prevent placing data centers at both
        # I wonder if the below step is contributing to the computational bottleneck. But we don't have to worry about it if it runs
        # well on your computer locally :) - RICHARD
        for (l1, l2), dist in location_distances.items():
            if dist < min_distance:
                for d1 in range(len(baseline_centers)):
                    for d2 in range(d1 + 1, len(baseline_centers)):
                        model.addConstr(x[d1, l1] + x[d2, l2] <= 1, f"min_dist_{d1}_{l1}_{d2}_{l2}")

    model.optimize()

    if model.status == GRB.OPTIMAL:
        print(f"Optimal solution found w/ objective value: {model.objVal}")

        # extract the solution
        solution = []
        for d in range(len(baseline_centers)):
            dc = baseline_centers[d]
            for l in location_ids:
                if x[d, l].x > 0.5:
                    loc_info = next(loc for loc in locations if loc["id"] == l)
                    solution.append({
                        "data_center_id": d,
                        "original_city": dc["city"],
                        "original_lat": dc["lat"],
                        "original_lng": dc["lng"],
                        "new_lat": loc_info["lat"],
                        "new_lng": loc_info["lng"],
                        "water_stress": loc_info["water_stress"],
                        "electricity_price": loc_info["electricity_price"],
                        "carbon_emissions": loc_info["carbon_emissions"],
                        "water_efficiency": loc_info["water_efficiency"],
                        "distance": calc_distance(
                            dc["lat"], dc["lng"],
                            loc_info["lat"], loc_info["lng"]
                        )
                        # feel free to adjujst this list, i was just making sure things seemed okay
                    })
                    break


        # LEFT OFF HERE - RICHARD

        baseline_metrics = []
        for dc in baseline_centers:
            min_dist = float('inf')
            closest_loc = None

            for loc in locations:
                dist = calc_distance(dc["lat"], dc["lng"], loc["lat"], loc["lng"])
                if dist < min_dist:
                    min_dist = dist
                    closest_loc = loc

            baseline_metrics.append({
                "city": dc["city"],
                "lat": dc["lat"],
                "lng": dc["lng"],
                "water_stress": closest_loc["water_stress"],
                "electricity_price": closest_loc["electricity_price"],
                "carbon_emissions": closest_loc["carbon_emissions"],
                "water_efficiency": closest_loc["water_efficiency"]
            })

        # just to see the average metrics
        baseline_avg = {
            "water_stress": sum(b["water_stress"] for b in baseline_metrics) / len(baseline_metrics),
            "electricity_price": sum(b["electricity_price"] for b in baseline_metrics) / len(baseline_metrics),
            "carbon_emissions": sum(b["carbon_emissions"] for b in baseline_metrics) / len(baseline_metrics),
            "water_efficiency": sum(b["water_efficiency"] for b in baseline_metrics) / len(baseline_metrics)
        }

        optimized_avg = {
            "water_stress": sum(s["water_stress"] for s in solution) / len(solution),
            "electricity_price": sum(s["electricity_price"] for s in solution) / len(solution),
            "carbon_emissions": sum(s["carbon_emissions"] for s in solution) / len(solution),
            "water_efficiency": sum(s["water_efficiency"] for s in solution) / len(solution),
            "avg_distance": sum(s["distance"] for s in solution) / len(solution)
        }

        # here are the metrics compared to the original ben chmark
        print("\nMetrics Comparison:")
        print(f"Water Stress: {baseline_avg['water_stress']:.2f} → {optimized_avg['water_stress']:.2f} " +
              f"({((baseline_avg['water_stress'] - optimized_avg['water_stress']) / baseline_avg['water_stress'] * 100):.2f}% improvement)")

        print(f"Electricity Price: ${baseline_avg['electricity_price']:.4f}/kWh → ${optimized_avg['electricity_price']:.4f}/kWh " +
              f"({((baseline_avg['electricity_price'] - optimized_avg['electricity_price']) / baseline_avg['electricity_price'] * 100):.2f}% improvement)")

        print(f"Carbon Emissions: {baseline_avg['carbon_emissions']:.4f} tons CO2/MWh → {optimized_avg['carbon_emissions']:.4f} tons CO2/MWh " +
              f"({((baseline_avg['carbon_emissions'] - optimized_avg['carbon_emissions']) / baseline_avg['carbon_emissions'] * 100):.2f}% improvement)")

        print(f"Water Efficiency: {baseline_avg['water_efficiency']:.2f} → {optimized_avg['water_efficiency']:.2f} " +
              f"({((optimized_avg['water_efficiency'] - baseline_avg['water_efficiency']) / baseline_avg['water_efficiency'] * 100):.2f}% improvement)")

        print(f"Average Relocation Distance: {optimized_avg['avg_distance']:.2f} km")

        return {
            "objective_value": model.objVal,
            "solution": solution,
            "baseline": baseline_metrics,
            "metrics": {
                "baseline": baseline_avg,
                "optimized": optimized_avg
            }
        }
    else:
        print(f"No optimal solution found. Status: {model.status}")
        return None



# \main execution
if __name__ == "__main__":
  #grid resoljution
    resolution = 0.25  # 0.25 degrees = ~28 km

    # adjust the spacing based on longitude (since longitude degrees are smaller at higher latitudes)
    # again, this is from my old UROP, so could be slightly off
    avg_lat = (AZ_MIN_LAT + AZ_MAX_LAT) / 2
    lng_spacing = resolution / np.cos(np.radians(avg_lat))

    # create baseline data centers (129 total for az)
    baseline_centers = create_baseline_centers()
    print(f"Created {len(baseline_centers)} baseline data centers")

    # make a grid of potential locations
    potential_locations = create_location_grid(resolution)
    print(f"Created {len(potential_locations)} potential locations")

    # minimum distance between data centers (in km)
    # just set it to 0 to disable this constraint
    min_distance = 20  # km

    results = optimize_data_center_locations(baseline_centers, potential_locations, min_distance)

Created 129 baseline data centers
Created 480 potential locations
Starting optimization with 129 data centers and 480 potential locations
Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads



GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information

In [ ]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 21.6 MB/s eta 0:00:00
